In [1]:
from numba import cuda
import cupy as cp

In [2]:
import numpy as np

In [3]:
@cuda.jit
def matmul(A, B, C):
    """Perform square matrix multiplication of C = A * B
    """
    i, j = cuda.grid(2)  
    if i < C.shape[0] and j < C.shape[1]:   # grid can extend beyond C
        tmp = 0.                            
        for k in range(A.shape[1]):
            tmp += A[i, k] * B[k, j]        # multiply elements in row i of A and column j of B and add to temp
        C[i, j] = tmp

In [4]:
print(cuda.detect())

Found 1 CUDA devices
id 0    b'NVIDIA GeForce GTX 1050 Ti'                              [SUPPORTED]
                      Compute Capability: 6.1
                           PCI Device ID: 0
                              PCI Bus ID: 1
                                    UUID: GPU-339f9d27-2e80-2bd6-56f6-04cdd9957792
                                Watchdog: Enabled
                            Compute Mode: WDDM
             FP32/FP64 Performance Ratio: 32
Summary:
	1/1 devices are supported
True


In [5]:
array_cpu = np.random.randint(0, 10, size = (2000, 2000))

In [6]:
d_array = cuda.to_device(array_cpu)

In [7]:
d_array # array on gpu

In [8]:
another = cp.asarray(d_array)

In [9]:
another # wrap around numpy array in gpu with cupy

array([[4, 3, 1, ..., 7, 0, 3],
       [8, 5, 3, ..., 0, 5, 0],
       [1, 5, 6, ..., 3, 1, 6],
       ...,
       [3, 5, 7, ..., 7, 8, 1],
       [2, 2, 4, ..., 5, 8, 6],
       [0, 4, 7, ..., 7, 1, 0]])

In [10]:
cp.random.seed(42)
A = cp.random.uniform(1, 10, size=(2001, 2001), dtype=np.float64)  # array 1
B = cp.random.uniform(1, 10, size=(2001, 2001), dtype=np.float64)  # array 2
C = cp.zeros((2001, 2001), dtype=np.float64)       # array where we store answer 

In [11]:
threadsperblock = (16, 16)  # each block will contain 16x16 threads, typically 128 - 512 threads/block
blockspergrid_x = int(np.ceil(C.shape[0] / threadsperblock[0]))
blockspergrid_y = int(np.ceil(C.shape[1] / threadsperblock[1]))
blockspergrid = (blockspergrid_x, blockspergrid_y)  # we calculate the gridsize (number of blocks) from array
print(blockspergrid)
print(f"The kernel will be executed up to element {threadsperblock[0]*blockspergrid_x}")

(126, 126)
The kernel will be executed up to element 2016


In [12]:
# execution of the kernel
import time
matmul[blockspergrid, threadsperblock](A, B, C)

In [13]:
C

array([[58625.02928708, 59143.63369398, 59095.4393774 , ...,
        58748.58966473, 59508.98777125, 60390.12590796],
       [59254.89751363, 60014.73151918, 59902.1798702 , ...,
        59219.47931994, 60405.08361905, 60661.79188881],
       [61427.53549638, 62078.04759424, 61537.45049997, ...,
        61259.85846596, 61901.11632625, 62373.99706587],
       ...,
       [60345.62552455, 61321.68749272, 61307.30625432, ...,
        60401.77517458, 61438.73915912, 62055.67262768],
       [60160.285775  , 60013.25117571, 60666.77857374, ...,
        59702.69675176, 60331.94483091, 61008.69645485],
       [60174.50688736, 60613.04591342, 60952.74040959, ...,
        59596.74842436, 60851.01807623, 60774.01029692]])

In [14]:
@cuda.jit
def matmul(A, B, C):
    """Perform square matrix multiplication of C = A * B
    """
    i, j = cuda.grid(2)  
    if i < C.shape[0] and j < C.shape[1]:   # grid can extend beyond C
        tmp = 0.                            
        for k in range(A.shape[1]):
            tmp += A[i, k] * B[k, j]        # multiply elements in row i of A and column j of B and add to temp
        C[i, j] = tmp

def powerMatrix(matrix, n, bpg = blockspergrid, tpb = threadsperblock):
    if n < 1:
        raise ValueError
        return None

    matrix1, matrix2 = matrix, matrix
    matrix3 = cp.zeros((len(matrix[0]), len(matrix[1])), dtype=np.float64)  

    for i in range(n-1):
        matmul[bpg, tpb](matrix1, matrix2, matrix3) # Sequential Part
        matrix1 = matrix3

    return matrix1

In [15]:
start = time.time()
powerMatrix(A, n = 10, bpg = blockspergrid, tpb = threadsperblock)
end = time.time()

In [16]:
print(f"{end - start:.4f} detik")

4.6740 detik
